In [ ]:
from collections import Counter
import glob
import numpy as np
import pandas as pd

### Analyze Results

In [ ]:
# TODO: change to "final"

sim_files = glob.glob(pathname="simulation_results/simulation*.csv")
n_sims = len(sim_files)
simulation_results = np.empty((n_sims, 20), dtype=object)

for i, file in enumerate(sim_files):
    record = pd.read_csv(file)
    placements = pd.Series(record['Result'])
    simulation_results[i] = placements.to_numpy()

In [ ]:
# Two-dimensional numpy array of simulation results 
# with each row as a simulation, each column as country, entry as placement 
# team index is same as in teams_list, starting with India as 0

with open('teams.txt') as file:
    teams_list = file.read().splitlines()

# Ex: Print results for all countries
for i in range(0,20):
    print(teams_list[i], "results:", dict(Counter(simulation_results[:,i])))

In [ ]:
# match_records contains all match results from the simulation set.
# This way, we can easily calculate the proportion of matches won
# by one team versus another across the whole simulation set.
# The outcome column is included only in the rare case a tie occurs.
# Below, the match records from the first simulation are printed.
match_records = pd.read_csv('match_records.csv')
match_records[['winner','loser','outcome']].head(55)

In [ ]:
# TODO: Visualize results based on simulation_results array and match_records dataframe

In [ ]:
# def result_bar(team):

In [ ]:
# def result_pie(team):

In [ ]:
def pair_results(teamA, teamB, visualize=False):
    if teamA == teamB:
        return
    num_victories = len(match_records.loc[(match_records['winner'] == teamA) &
                                           (match_records['loser'] == teamB) &
                                           (match_records['outcome'] == 'w')])
    num_total = len(match_records.loc[((match_records['winner'] == teamA) &
                                           (match_records['loser'] == teamB)) |
                                           ((match_records['loser'] == teamA) &
                                           (match_records['winner'] == teamB))])
    if num_total == 0:
        print(f"{teamA} did not play {teamB} in this simulation set.")
    else:
        prop = num_victories/num_total
        print(f"{teamA} beat {teamB} {num_victories} times in {num_total} matches, for a winning proportion of {prop}.")

In [ ]:
# Get bar chart / pie chart for any team here!
team = 'Australia'  # Example
# result_bar(team)
# result_pie(team)

In [ ]:
# Get proportion of matches teamA won vs teamB
# Set visualize to True if desired
teamA = 'West Indies'
for team in teams_list:    
    teamB = team
    pair_results(teamA, team)

In [ ]:
# Bar / pie charts for results could be good
# pair_results could also work (especially if all shown for given team)
# What else? Incorporate points or NRR? Something is missing with just these two